In [2]:
import numpy as np
import rasterio 
from PIL import Image
import cv2
import os

In [3]:
batch_size = 128
path = os.path.join("C:\\Users\\andra\\guorbit\\dataset", "e_tr", "image")
np_path = os.path.join("C:\\Users\\andra\\guorbit\\batch_dataset", "b_tr", "image")
files_to_load = sorted(os.listdir(path))[0:batch_size]
path_to_files = [os.path.join(path, file) for file in files_to_load]
print(path_to_files)

['C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0001.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0002.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0003.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0004.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0005.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0006.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0007.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0008.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0009.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0010.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0011.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0012.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0013.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0014.tif', 'C:\\Users\\andra\\guorbit\\dataset\\e_tr\\image\\O10_0015.tif', 'C:\\Users\\andra\\guorb

In [3]:
%%timeit -n 10
# Open the raster file
for i in path_to_files:
    with rasterio.open(i) as file:
        image = file.read()


c:\Users\andra\anaconda3\envs\land_class\lib\site-packages\rasterio\__init__.py:319: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


1.32 s ± 86.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%%timeit -n 10
#open files with cv2
for i in path_to_files:
    image = cv2.imread(i)

2.7 s ± 981 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%timeit -n 10
#open files with PIL
for i in path_to_files:
    image = Image.open(i)

217 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:

#open files with numpy
batch = np.load(os.path.join(np_path, "0.npy"))


In [8]:
batch_path = os.path.join(np_path, "0.npy")

In [17]:
%%timeit -n 10
#open files with numpy
batch = np.load(batch_path)

72.4 ms ± 2.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
from concurrent.futures import ThreadPoolExecutor
import rasterio

In [49]:
%%timeit

def read_file(file_path):
    with rasterio.open(file_path) as file:
        image = file.read()
        # Do something with the image
#path_to_files = ["file1.tif", "file2.tif", "file3.tif"]  # Replace with your actual file paths

with ThreadPoolExecutor() as executor:
    executor.map(read_file, path_to_files)




1.13 s ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%%timeit
def read_file(file_path):
    image = cv2.imread(file_path)
    # Do something with the image


with ThreadPoolExecutor() as executor:
    executor.map(read_file, path_to_files)
        

# Measure the time taken for 100 iterations



433 ms ± 20 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
%%timeit
def read_file(file_path):
    image = Image.open(file_path)
    # Do something with the image

with ThreadPoolExecutor() as executor:
    executor.map(read_file, path_to_files)

109 ms ± 9.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
batch_paths = [os.path.join(np_path, str(i)+".npy") for i in range(20)]

In [6]:
import asyncio
import aiofiles
from io import BytesIO

In [7]:
%%timeit
def read_file(file_path):
    batch = np.load(file_path)
    # Do something with the image
    return batch

results = []

with ThreadPoolExecutor() as executor:
    results = list(executor.map(read_file, batch_paths))


print(np.array(results).shape)


(20, 128, 512, 512, 4)
(20, 128, 512, 512, 4)
(20, 128, 512, 512, 4)
(20, 128, 512, 512, 4)
(20, 128, 512, 512, 4)
(20, 128, 512, 512, 4)
(20, 128, 512, 512, 4)
(20, 128, 512, 512, 4)
1.86 s ± 413 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:


async def read_file_async(file_path):
    async with aiofiles.open(file_path, mode='rb') as f:
        content = await f.read()
        batch = np.load(BytesIO(content))
    return batch

async def main(batch_paths):
    tasks = []
    for path in batch_paths:
        task = asyncio.ensure_future(read_file_async(path))
        tasks.append(task)

    results = await asyncio.gather(*tasks)
    return np.array(results)

results = await main(batch_paths)
print(results.shape)


(4, 128, 4, 512, 512)
